In [1]:
from psd_tools import PSDImage
import os

# --- 请将这里的 'template.psd' 替换成您自己的PSD文件名 ---
TEMPLATE_PSD_PATH = r'E:/github/psd2print/男装短袖模板/男装短袖模板/z男装短袖版-M.psd'

if not os.path.exists(TEMPLATE_PSD_PATH):
    print(f"错误: 模板文件 '{TEMPLATE_PSD_PATH}' 不存在！请将您的PSD文件名更新到代码中。")
    exit()

print(f"--- 开始深度分析PSD文件: {TEMPLATE_PSD_PATH} ---")
print("="*50)

try:
    psd = PSDImage.open(TEMPLATE_PSD_PATH)

    if len(psd) == 0:
        print("警告：这个PSD文件里没有任何图层。")
    
    # 遍历文件中的每一个图层，不做任何过滤
    for i, layer in enumerate(psd):
        print(f"图层序号: {i}")
        print(f"  - 图层名称 (Name): '{layer.name}'")
        print(f"  - 图层类型 (Kind): '{layer.kind}'") # <--- 这是最重要的信息！
        print(f"  - 是否可见 (Visible): {layer.is_visible()}")
        print(f"  - 是否为组 (Group): {layer.is_group()}")
        print(f"  - 尺寸 (W x H): {layer.width} x {layer.height}")
        print(f"  - 位置 (L, T): {layer.left}, {layer.top}")

        # 尝试渲染图层，看是否能成功生成图像
        try:
            composite_image = layer.composite()
            if composite_image:
                print(f"  - 渲染测试 (Composite): 成功 (生成图像尺寸 {composite_image.width}x{composite_image.height})")
            else:
                # 这通常发生在图层完全不可见或为空的情况下
                print(f"  - 渲染测试 (Composite): 失败或为空")
        except Exception as e:
            print(f"  - 渲染测试 (Composite): 发生错误 - {e}")
            
        print("-"*25)

except Exception as e:
    print(f"\n打开或分析PSD文件时发生严重错误: {e}")

print("="*50)
print("诊断完毕。")

Unknown image resource 1092


--- 开始深度分析PSD文件: E:/github/psd2print/男装短袖模板/男装短袖模板/z男装短袖版-M.psd ---
图层序号: 0
  - 图层名称 (Name): '组 2'
  - 图层类型 (Kind): 'group'
  - 是否可见 (Visible): True
  - 是否为组 (Group): True
  - 尺寸 (W x H): 6204 x 3183
  - 位置 (L, T): 34, 19
  - 渲染测试 (Composite): 成功 (生成图像尺寸 6204x3183)
-------------------------
图层序号: 1
  - 图层名称 (Name): '组 1'
  - 图层类型 (Kind): 'group'
  - 是否可见 (Visible): True
  - 是否为组 (Group): True
  - 尺寸 (W x H): 3967 x 3174
  - 位置 (L, T): 1145, 24
  - 渲染测试 (Composite): 成功 (生成图像尺寸 3967x3174)
-------------------------
诊断完毕。


In [2]:
import os
from psd_tools import PSDImage

# --- 配置区域 ---
# 您要检测的PSD文件的完整路径
# 使用 r'...' 来处理Windows路径，避免反斜杠问题
TEMPLATE_PSD_PATH = r'E:/github/psd2print/男装短袖模板/男装短袖模板/z男装短袖版-L.psd'

# --- 功能函数定义 ---
def find_all_renderable_layers(layer_source, found_layers_list):
    """
    递归函数，用于查找所有可用的、非组的图层。
    它可以“钻入”图层组内部。
    """
    for layer in layer_source:
        if layer.is_group():
            # 如果是图层组，就继续向内查找
            find_all_renderable_layers(layer, found_layers_list)
        else:
            # 如果是普通图层，检查它是否可见且有内容
            if layer.is_visible() and layer.width > 0 and layer.height > 0:
                found_layers_list.append(layer)

# --- 主程序入口 ---
if __name__ == "__main__":
    # 检查文件是否存在
    if not os.path.exists(TEMPLATE_PSD_PATH):
        print(f"错误: 文件不存在！\n路径: '{TEMPLATE_PSD_PATH}'")
        exit()

    try:
        filename = os.path.basename(TEMPLATE_PSD_PATH)
        print(f"--- 正在分析文件: {filename} ---")

        # 打开PSD文件
        psd = PSDImage.open(TEMPLATE_PSD_PATH)

        # 查找所有可用图层
        all_layers = []
        find_all_renderable_layers(psd, all_layers)

        # 打印结果
        if not all_layers:
            print("\n结果: 在此文件中没有找到任何可见的、有内容的图层。")
        else:
            print(f"\n结果: 在 '{filename}' 中共找到 {len(all_layers)} 个可用图层，名称如下：")
            print("-" * 30)
            for layer in all_layers:
                print(f"- {layer.name}")
            print("-" * 30)

    except Exception as e:
        print(f"\n处理文件时发生错误: {e}")

Unknown image resource 1092


--- 正在分析文件: z男装短袖版-L.psd ---

结果: 在 'z男装短袖版-L.psd' 中共找到 10 个可用图层，名称如下：
------------------------------
- 领
- 右袖
- 左袖
- 后
- 前
- 右L
- L
- L
- L
- 左L
------------------------------


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from psd_tools import PSDImage

# --- 全局配置区域 ---
TEMPLATE_PSD_DIR = 'E:/github/psd2print/男装短袖模板/男装短袖模板'
PATTERN_FOLDER_PATH = 'E:/github/psd2print/WGTZW25042869/WGTZW25042869/'
output_dir = 'final_prints'

TARGET_LAYER_NAMES = ['领', '右袖', '左袖', '后片', '前片']
PATTERN_FILENAMES = ['03.jpg', '05.jpg', '04.jpg', '02.jpg', '01.jpg']

# --- 功能函数定义 ---

def find_all_renderable_layers(layer_source, found_layers):
    """递归函数，查找所有可用的、非组的图层。"""
    for layer in layer_source:
        if layer.is_group():
            find_all_renderable_layers(layer, found_layers)
        else:
            if layer.is_visible() and layer.width > 0 and layer.height > 0:
                found_layers.append(layer)

def apply_pattern_to_layer(layer, pattern_image):
    """将给定的印花图案应用到指定的图层形状上。"""
    # 这部分代码保持不变
    layer_pil = layer.composite()
    if layer_pil is None:
        print(f"  ❌ 警告: 图层 '{layer.name}' 为空，已跳过。")
        return None
    layer_cv = cv2.cvtColor(np.array(layer_pil), cv2.COLOR_RGBA2BGRA)
    try:
        b, g, r, alpha = cv2.split(layer_cv)
    except ValueError:
        print(f"  ❌ 错误: 图层 '{layer.name}' 的图像通道不是4通道RGBA，无法处理。")
        return None
    _, mask = cv2.threshold(alpha, 10, 255, cv2.THRESH_BINARY)
    pattern_cv = cv2.cvtColor(np.array(pattern_image), cv2.COLOR_RGB2BGR)
    h, w = mask.shape
    resized_pattern = cv2.resize(pattern_cv, (w, h))
    final_piece = cv2.bitwise_and(resized_pattern, resized_pattern, mask=mask)
    b, g, r = cv2.split(final_piece)
    final_piece_rgba = cv2.merge([b, g, r, mask])
    return final_piece_rgba

# [最终版本] 用这个函数完整替换您脚本里的旧函数
def add_label_to_piece(image_to_label, label_text, rolate=False):
    """在给定的图像上添加带描边的文字标签，并根据需要独立旋转文字。"""
    # --- 1. 样式与尺寸定义 ---
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    main_color_bgra = (0, 0, 255, 255)      # 红色字体 (B,G,R,A)
    outline_color_bgra = (255, 255, 255, 255) # 白色描边
    main_thickness = 2
    outline_thickness = 5

    if rolate:
        # --- [旋转文字的逻辑] ---
        
        # 1. 创建一个仅用于绘制文字的独立、透明的小画布
        (text_w, text_h), baseline = cv2.getTextSize(label_text, font, font_scale, main_thickness)
        # 留出一些边距，防止旋转后文字被裁剪
        canvas_h = text_h + baseline + 10
        canvas_w = text_w + 10
        text_canvas = np.zeros((canvas_h, canvas_w, 4), dtype=np.uint8)

        # 2. 在这个小画布上绘制水平的文字（描边+主体）
        # 注意，绘制的坐标是相对于这个小画布的
        text_org = (5, text_h + 5) # 在小画布上的绘制起点
        cv2.putText(text_canvas, label_text, text_org, font, font_scale, outline_color_bgra, outline_thickness, cv2.LINE_AA)
        cv2.putText(text_canvas, label_text, text_org, font, font_scale, main_color_bgra, main_thickness, cv2.LINE_AA)

        # 3. 将这个绘制好文字的小画布旋转180度
        rotated_text_canvas = cv2.flip(text_canvas, -1)

        # 4. 将旋转后的文字“贴”到原始裁片上
        # 计算粘贴位置：顶部居中
        img_h, img_w, _ = image_to_label.shape
        paste_x = (img_w - canvas_w) // 2
        paste_y = 30 # 从顶部向下30像素

        # 为了方便地处理带透明通道的粘贴，我们临时将两个图像都转为Pillow格式
        main_image_pil = Image.fromarray(cv2.cvtColor(image_to_label, cv2.COLOR_BGRA2RGBA))
        rotated_text_pil = Image.fromarray(cv2.cvtColor(rotated_text_canvas, cv2.COLOR_BGRA2RGBA))

        # 使用Pillow的paste方法，利用自身的alpha通道作为蒙版进行粘贴
        main_image_pil.paste(rotated_text_pil, (paste_x, paste_y), rotated_text_pil)
        
        # 将最终结果转回OpenCV格式并返回
        final_image = cv2.cvtColor(np.array(main_image_pil), cv2.COLOR_RGBA2BGRA)

    else:
        # --- [不旋转文字的逻辑] (保持原样) ---
        image_to_draw_on = image_to_label
        img_h, img_w, _ = image_to_draw_on.shape
        (text_w, text_h), baseline = cv2.getTextSize(label_text, font, font_scale, main_thickness)
        text_x = (img_w - text_w) // 2
        text_y = img_h - 30
        
        cv2.putText(image_to_draw_on, label_text, (text_x, text_y), font, font_scale, outline_color_bgra, outline_thickness, cv2.LINE_AA)
        cv2.putText(image_to_draw_on, label_text, (text_x, text_y), font, font_scale, main_color_bgra, main_thickness, cv2.LINE_AA)
        
        final_image = image_to_draw_on

    return final_image

def process_single_template(template_psd_path):
    """处理单个PSD模板文件的完整流程。"""
    filename = os.path.basename(template_psd_path)
    print(f"\n{'='*20} 开始处理模板: {filename} {'='*20}")

    # **[核心改动 1]** 从文件名中提取尺码标签
    try:
        base_name = os.path.splitext(filename)[0]  # 例如: 'z男装短袖版-M'
        size_label = base_name.split('-')[-1]     # 例如: 'M'
        print(f"  ℹ️ 从文件名提取到尺码标签: '{size_label}'")
    except IndexError:
        size_label = "N/A" # 如果文件名格式不符，则使用默认标签
        print(f"  ⚠️ 警告: 无法从文件名 '{filename}' 中提取尺码标签。")

    psd = PSDImage.open(template_psd_path)
    final_canvas = Image.new('RGBA', (psd.width, psd.height), (0, 0, 0, 0))

    all_layers = []
    find_all_renderable_layers(psd, all_layers)
    if not all_layers:
        print(f"警告：在模板 '{filename}' 中未找到任何可用的图层，已跳过。")
        return

    for target_name, pattern_filename in zip(TARGET_LAYER_NAMES, PATTERN_FILENAMES):
        full_pattern_path = os.path.join(PATTERN_FOLDER_PATH, pattern_filename)
        if not os.path.exists(full_pattern_path):
            print(f"警告: 印花图 '{full_pattern_path}' 不存在，已跳过处理图层 '{target_name}'。")
            continue

        found_layer = next((layer for layer in all_layers if layer.name == target_name), None)
        
        if found_layer:
            pattern_image = Image.open(full_pattern_path).convert("RGB")
            
            # 您原有的特殊旋转逻辑保持不变
            if filename == '男装短袖版-M.psd' and found_layer.name == '后片':
                pattern_image = pattern_image.rotate(180, expand=True)
                print(f"  🔄 特别处理: 为 '{filename}' 的 '{found_layer.name}' 图层应用了旋转180度的印花。")

            processed_image_cv = apply_pattern_to_layer(found_layer, pattern_image)
            
            if processed_image_cv is not None:
                # **[核心改动 2]** 在粘贴前，调用新函数添加标签
                if found_layer.name == '后片' and filename == '男装短袖版-M.psd':
                    image_with_label = add_label_to_piece(processed_image_cv, size_label,True)
                elif found_layer.name == '右袖' and filename == '男装短袖版-2XL.psd':
                    image_with_label = add_label_to_piece(processed_image_cv, size_label,True)
                elif found_layer.name == '左袖' and filename == '男装短袖版-2XL.psd':
                    image_with_label = add_label_to_piece(processed_image_cv, size_label,True)
                elif found_layer.name == '右袖' and filename == '男装短袖版-XL.psd':
                    image_with_label = add_label_to_piece(processed_image_cv, size_label,True)
                else:
                    image_with_label = add_label_to_piece(processed_image_cv, size_label)
                
                # 将最终处理好的（带标签的）图像转换为Pillow格式
                processed_image_pil = Image.fromarray(cv2.cvtColor(image_with_label, cv2.COLOR_BGRA2RGBA))
                
                paste_position = (found_layer.left, found_layer.top)
                final_canvas.paste(processed_image_pil, paste_position, processed_image_pil)
                print(f"  ✅ 已将带标签的 '{found_layer.name}' 裁片粘贴到最终画布上。")
        else:
            print(f"警告: 目标图层 '{target_name}' 在模板 '{filename}' 中未找到，已跳过。")

    output_path_name = os.path.splitext(filename)[0]
    final_output_path = os.path.join(output_dir, f"{output_path_name}.png")
    final_canvas.save(final_output_path)
    
    print(f"✅ 模板 '{filename}' 处理完毕，最终效果大图已保存至: {final_output_path}")

# --- 主程序入口 (这部分不变) ---
if __name__ == "__main__":
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    if not os.path.exists(TEMPLATE_PSD_DIR):
        print(f"错误: 模板文件夹 '{TEMPLATE_PSD_DIR}' 不存在！")
        exit()
    if len(TARGET_LAYER_NAMES) != len(PATTERN_FILENAMES):
        print("错误：目标图层列表和印花文件列表的数量不匹配！")
        exit()

    for filename in os.listdir(TEMPLATE_PSD_DIR):
        if filename.lower().endswith('.psd'):
            template_path = os.path.join(TEMPLATE_PSD_DIR, filename)
            try:
                process_single_template(template_path)
            except Exception as e:
                print(f"--- 严重错误: 处理文件 {filename} 时发生意外: {e} ---")

    print(f"\n{'='*25} 所有PSD文件处理完毕 {'='*25}")

Unknown image resource 1092



==================== 开始处理模板: 男装短袖版-2XL.psd ====================
  ℹ️ 从文件名提取到尺码标签: '2XL'
  ✅ 已将带标签的 '领' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '右袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '左袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '后片' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '前片' 裁片粘贴到最终画布上。


Unknown image resource 1092


✅ 模板 '男装短袖版-2XL.psd' 处理完毕，最终效果大图已保存至: final_prints\男装短袖版-2XL.png

==================== 开始处理模板: 男装短袖版-L.psd ====================
  ℹ️ 从文件名提取到尺码标签: 'L'
  ✅ 已将带标签的 '领' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '右袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '左袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '后片' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '前片' 裁片粘贴到最终画布上。


Unknown image resource 1092


✅ 模板 '男装短袖版-L.psd' 处理完毕，最终效果大图已保存至: final_prints\男装短袖版-L.png

==================== 开始处理模板: 男装短袖版-M.psd ====================
  ℹ️ 从文件名提取到尺码标签: 'M'
  ✅ 已将带标签的 '领' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '右袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '左袖' 裁片粘贴到最终画布上。
  🔄 特别处理: 为 '男装短袖版-M.psd' 的 '后片' 图层应用了旋转180度的印花。
  ✅ 已将带标签的 '后片' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '前片' 裁片粘贴到最终画布上。


Unknown image resource 1092


✅ 模板 '男装短袖版-M.psd' 处理完毕，最终效果大图已保存至: final_prints\男装短袖版-M.png

==================== 开始处理模板: 男装短袖版-S.psd ====================
  ℹ️ 从文件名提取到尺码标签: 'S'
  ✅ 已将带标签的 '领' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '右袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '左袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '后片' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '前片' 裁片粘贴到最终画布上。


Unknown image resource 1092


✅ 模板 '男装短袖版-S.psd' 处理完毕，最终效果大图已保存至: final_prints\男装短袖版-S.png

==================== 开始处理模板: 男装短袖版-XL.psd ====================
  ℹ️ 从文件名提取到尺码标签: 'XL'
  ✅ 已将带标签的 '领' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '右袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '左袖' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '后片' 裁片粘贴到最终画布上。
  ✅ 已将带标签的 '前片' 裁片粘贴到最终画布上。
✅ 模板 '男装短袖版-XL.psd' 处理完毕，最终效果大图已保存至: final_prints\男装短袖版-XL.png

========================= 所有PSD文件处理完毕 =========================
